In [2]:

#  This is my method ..
#

%xmode Minimal

#  Setting display options 

import pandas as pd
import numpy  as np
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate


print("--")


Exception reporting mode: Minimal


--


In [3]:

#  This is my method ..
#

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#########################################################################


In [4]:


%load_ext autoreload
%autoreload 2


from katana import remote
#  from katana.remote import export_data, import_data


my_client = remote.Client()
print(my_client.server_version)

print()
print("--")



0.9.0+20230315T145956Z.dbe1b6788.dev

--


In [5]:

from config import hyperparams


input_config = hyperparams.load_input_config()
print(input_config)


print()
print("--")



DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
InputConfig:
    num_partitions = 3,
    test_size = 0.05,
    random_state = 42

--


In [6]:

#  This is my method ..
#

#     from timeit import default_timer
#     
#     # Import the module that uses Dask to import drug data
#     from src import dask_ingestion
#     
#     graph = client.create_graph(num_partitions=input_config.num_partitions)


#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#     if input_config.use_train_rdg:
#         print(f"Import pretrained graph from: {input_config.trained_rdg_path}")
#         import_data.rdg(graph, input_config.trained_rdg_path)
#     else:
#         print("Generate the graph with data from source")
#         dask_ingestion.generate_deepcdr_graph(graph, input_config)



<_Graph my_graph, 974BNYMBeYB9K781hP55Qqpn2UXUpm6MN1EeV3Q2w99Y, 2>


In [7]:


from src import katana_pipeline

#  This method returns a handle only, no processing ..
#
rec_pipeline = katana_pipeline.RecipePipeline(my_graph)


#  rec_pipeline.graph.schema().view()



This ran MMM 01


In [ ]:

#########################################################################


In [8]:

#  Original code ..
#
#  start_time = default_timer()
#  
rec_pipeline.feature_generator()
#  
#  print(f"***Took {default_timer() - start_time} seconds to generate the features.***")


   ##########################################################


#  feature_generator() runs,
#
#       features = {
#           "HAS_EXPRESSION_OBSERVATION": "genomics_expression",
#           "HAS_METHYLATION_OBSERVATION": "genomics_methylation",
#           "HAS_MUTATION_OBSERVATION": "genomics_mutation",
#       }
#       for u, v in features.items():
#           preprocessing.set_cell_line_property(self.graph, u, v)
#
#       preprocessing.remove_null_cells(self.graph)
#       preprocessing.delete_single_node(self.graph)

#  And here are those,
#
#     def set_cell_line_property(graph, edge_label: str, genomics_property: str):
#         graph.query(
#             f"""
#             MATCH (c:CELL_LINE)-[r:{edge_label}]->(g:GENE)
#             WITH c, r, g
#             ORDER by g.ID
#             WITH c, collect(r.observation) as {genomics_property}
#             SET c.{genomics_property} = {genomics_property}
#             """
#         )
#     
#     
#     def delete_single_node(graph):
#         """Delete single node in a graph"""
#         graph.query(
#             """
#             MATCH (n)
#             WHERE NOT (n)--()
#             DELETE n
#             """
#         )
#     
#     
#     def remove_null_cells(graph):
#         """Delete cells node with NULL genomics features"""
#         graph.query(
#             """
#             MATCH (c:CELL_LINE)
#             WHERE c.genomics_methylation IS NULL
#                 OR c.genomics_expression IS NULL
#                 OR c.genomics_mutation IS NULL
#             DETACH DELETE c
#             """
#         )


   ##########################################################
    
    
#  So basically step 1 should add a CELL_LINE property
#
#  Here it is pre,
#
#     +---------------+------------------------------+-------+
#     | label         | my_keys                      |   cnt |
#     |---------------+------------------------------+-------|
#     | ['CELL_LINE'] | ['id', 'label']              |   240 |
#     | ['CELL_LINE'] | ['id', 'label', 'tcga_code'] |  1217 |
#
#     | ['DRUG']      | ['id', 'label', 'smiles']    |   223 |
#     | ['GDSC']      | ['id', 'label']              |   266 |
#     | ['GENE']      | ['id']                       |   719 |
#     +---------------+------------------------------+-------+
#
#  And here it is post,
#
#     Collecting genomics_expression features...
#     Collecting genomics_methylation features...
#     Collecting genomics_mutation features...
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['id', 'label']                                                                                  |   218 |
#     | ['CELL_LINE'] | ['id', 'label', 'tcga_code']                                                                     |   678 |
#
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'id', 'label']                                   |     8 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'id', 'label', 'tcga_code']                      |   195 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#   
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+


   ##########################################################
    
    
#  Step 2 should delete disconnected nodes ..
#
#  Here it is pre,
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#
#     | ['CELL_LINE'] | ['id', 'label']                                                                                  |   218 |
#     | ['CELL_LINE'] | ['id', 'label', 'tcga_code']                                                                     |   678 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'id', 'label']                                   |     8 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'id', 'label', 'tcga_code']                      |   195 |
#
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#
#  And here it is post,
#
#     Deleting Cell lines with NULL features...
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+


   ##########################################################
    
    
#  And step 3, 
#
#  Here it is pre,
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     
#     +-----------------------------+-------------------------------------+----------+
#     | my_type                     | my_keys                             |      cnt |
#     |-----------------------------+-------------------------------------+----------|
#     | FOR_DRUG                    | ['TYPE']                            |      476 |
#     | HAS_CELL_LINE               | ['TYPE']                            |    37400 |
#     | HAS_EXPRESSION_OBSERVATION  | ['TYPE', 'observation']             |   499052 |
#     | HAS_METHYLATION_OBSERVATION | ['TYPE', 'observation']             |   545560 |
#     | HAS_MUTATION_OBSERVATION    | ['TYPE', 'mutation', 'observation'] | 24776358 |
#     | HAS_CELL_LINE               | ['TYPE', 'label']                   |   153056 |
#     +-----------------------------+-------------------------------------+----------+
#
#  And here it is post,       (no change)
#
#      Deleting Cell lines with NULL features...
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     
#     +-----------------------------+-------------------------------------+----------+
#     | my_type                     | my_keys                             |      cnt |
#     |-----------------------------+-------------------------------------+----------|
#     | FOR_DRUG                    | ['TYPE']                            |      476 |
#     | HAS_CELL_LINE               | ['TYPE']                            |    37400 |
#     | HAS_EXPRESSION_OBSERVATION  | ['TYPE', 'observation']             |   499052 |
#     | HAS_METHYLATION_OBSERVATION | ['TYPE', 'observation']             |   545560 |
#     | HAS_MUTATION_OBSERVATION    | ['TYPE', 'mutation', 'observation'] | 24776358 |
#     | HAS_CELL_LINE               | ['TYPE', 'label']                   |   153056 |
#     +-----------------------------+-------------------------------------+----------+



          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

Deleting Cell lines with NULL features...


          0/? [?op/s]

Deleting single nodes


          0/? [?op/s]

In [ ]:

#  This is read only ..
#

#  stats = rec_pipeline.stats()
#  stats


In [ ]:


#  Original code
#

#  start_time = default_timer()

rec_pipeline.split_generator(input_config)

#  print(f"***Took {default_timer() - start_time} seconds to generate the split.***")
#  rec_pipeline.graph.schema().view()


   ##########################################################

    
#  split_generator() runs ..
#
#       print("Deleting old split")
#       split.delete_split(self.graph)
#
#       print("Generating split")
#       split_df = pandas.concat(
#           self.graph.query(
#               """
#           MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
#           RETURN u.label as label, c.tcga_code as tcga_code,
#           a.id as drug_id, c.id as cell_line_id
#           """
#           )
#       )
#       split_df = split_df.sort_values(by=["drug_id", "cell_line_id", "label"]).reset_index(drop=True)
#       df_split = split.generate_split(split_df, input_hp.random_state, input_hp.test_size)
#
#       print("Upsert split on the graph")
#       split.upsert_split(self.graph, df_split)

#  And  here are those,
#
#     class SplitType(IntEnum):
#         """SplitType encodes training, validation, and test data separation"""
#     
#         TRAIN = 0
#         VAL = 1
#         TEST = 2
#     
#     def delete_split(graph):
#         graph.query(
#             """
#             MATCH (:DRUG)-[u:SPLIT]->(:CELL_LINE)
#             DELETE u
#             """
#         )
#     
#     def generate_split(df, random_state, test_size):
#         df_train, df_test = train_test_split(
#             df, test_size=test_size, random_state=random_state, stratify=df["tcga_code"], shuffle=True,
#         )
#         df_train["split"], df_test["split"] = SplitType.TRAIN, SplitType.VAL
#         df_split = pandas.concat([df_train, df_test], ignore_index=True)
#         df_split = df_split.astype({"drug_id": "string", "cell_line_id": "string"})
#         return df_split.drop(columns=["tcga_code"])
#     
#     
#     def upsert_split(graph, split):
#         with DataFrameImporter(graph) as df_importer:
#             df_importer.node_id_property_name("id")
#             df_importer.edges_dataframe(
#                 split,
#                 source_id_space="DRUG",
#                 destination_id_space="CELL_LINE",
#                 source_column="drug_id",
#                 destination_column="cell_line_id",
#                 type="SPLIT",
#             )
#             df_importer.insert()

    
   ##########################################################


#  Here is just step 1, delete_split(), pre,     (this should be a no-op) ..
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     
#     +-----------------------------+-------------------------------------+----------+
#     | my_type                     | my_keys                             |      cnt |
#     |-----------------------------+-------------------------------------+----------|
#     | FOR_DRUG                    | ['TYPE']                            |      476 |
#     | HAS_CELL_LINE               | ['TYPE']                            |    37400 |
#     | HAS_EXPRESSION_OBSERVATION  | ['TYPE', 'observation']             |   499052 |
#     | HAS_METHYLATION_OBSERVATION | ['TYPE', 'observation']             |   545560 |
#     | HAS_MUTATION_OBSERVATION    | ['TYPE', 'mutation', 'observation'] | 24776358 |
#     | HAS_CELL_LINE               | ['TYPE', 'label']                   |   153056 |
#     +-----------------------------+-------------------------------------+----------+
#
#  And post,   (no change) ..
#
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     | label         | my_keys                                                                                          |   cnt |
#     |---------------+--------------------------------------------------------------------------------------------------+-------|
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label', 'tcga_code'] |   344 |
#     | ['DRUG']      | ['id', 'label', 'smiles']                                                                        |   223 |
#     | ['GDSC']      | ['id', 'label']                                                                                  |   266 |
#     | ['CELL_LINE'] | ['genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'label']              |    14 |
#     | ['GENE']      | ['id']                                                                                           |   719 |
#     +---------------+--------------------------------------------------------------------------------------------------+-------+
#     
#     +-----------------------------+-------------------------------------+----------+
#     | my_type                     | my_keys                             |      cnt |
#     |-----------------------------+-------------------------------------+----------|
#     | FOR_DRUG                    | ['TYPE']                            |      476 |
#     | HAS_CELL_LINE               | ['TYPE']                            |    37400 |
#     | HAS_EXPRESSION_OBSERVATION  | ['TYPE', 'observation']             |   499052 |
#     | HAS_METHYLATION_OBSERVATION | ['TYPE', 'observation']             |   545560 |
#     | HAS_MUTATION_OBSERVATION    | ['TYPE', 'mutation', 'observation'] | 24776358 |
#     | HAS_CELL_LINE               | ['TYPE', 'label']                   |   153056 |
#     +-----------------------------+-------------------------------------+----------+


   ##########################################################

    
#  Just step 2, 
#
#     This runs a Cypher, takes that result set, and continues processing.
#     We'll put just this Cypher in the next cell
#
#  Sample output
#
#     85024
#
#     +-----------+-------------+-----------+----------------+
#     |     label | tcga_code   |   drug_id | cell_line_id   |
#     |-----------+-------------+-----------+----------------|
#     | -0.587494 | KIRC        |   6450816 | ACH-000046     |
#     | -5.71602  | OV          |   6450816 | ACH-000048     |
#     +-----------+-------------+-----------+----------------+


   ##########################################################
    
    
#  Just step 3,
#
#     .  Sorts the dataframe produces above. 



In [11]:

#  This is just step 2 from the cell above, a Cypher ..
#

l_query  = """
   MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
   RETURN u.label as label, c.tcga_code as tcga_code,
   a.id as drug_id, c.id as cell_line_id
   """.format()
      #
l_result = my_graph.query_unpaginated(l_query)

print(len(l_result))
print(tabulate(l_result.head(2), headers = "keys", tablefmt = "psql", showindex = False))
   #
print("--")

#  Sample output
#
#     85024
#
#     +-----------+-------------+-----------+----------------+
#     |     label | tcga_code   |   drug_id | cell_line_id   |
#     |-----------+-------------+-----------+----------------|
#     | -0.587494 | KIRC        |   6450816 | ACH-000046     |
#     | -5.71602  | OV          |   6450816 | ACH-000048     |
#     +-----------+-------------+-----------+----------------+



          0/? [?op/s]

68638
+----------+-------------+-----------+----------------+
|    label | tcga_code   |   drug_id | cell_line_id   |
|----------+-------------+-----------+----------------|
| 0.302727 | BLCA        |   6918848 | ACH-000018     |
| 0.840222 | PAAD        |   6918848 | ACH-000023     |
+----------+-------------+-----------+----------------+
--


In [12]:

#  After the above, we sort ..
#
#  Original line below ..
#
#     split_df = split_df.sort_values(by=["drug_id", "cell_line_id", "label"]).reset_index(drop=True)


l_result = l_result.sort_values(by=["drug_id", "cell_line_id", "label"]).reset_index(drop=True)

print("--")

   ###
    
#  And then we run this,
#
#  Original line below ..
#
#      df_split = split.generate_split(split_df, input_hp.random_state, input_hp.test_size)

print(input_hp.random_state)


l_result = split.generate_split(l_result, input_hp.random_state, input_hp.test_size)









--


In [ ]:

#########################################################################


In [ ]:
model_config = hyperparams.load_model_config()
model_config

In [ ]:
training_config = hyperparams.load_training_config()
training_config

In [ ]:
start_time = default_timer()
validation_metric = rec_pipeline.train(model_config, training_config)
print(f"***Took {default_timer() - start_time} seconds to train the model.***")
print("Validation metric: ", validation_metric)

In [ ]:
start_time = default_timer()
metrics, predictions, labels = rec_pipeline.test(training_config)
print(f"***Took {default_timer() - start_time} seconds to test the model.***")
metrics

In [ ]:
start_time = default_timer()
rec_pipeline.plot(labels, predictions)
print(f"***Took {default_timer() - start_time} seconds to plot figures.***")

In [ ]:
rec_pipeline.infer(training_config)

In [ ]:
bortezomib = "B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)(O)O"
cell_line = "ACH-000001"
rec_pipeline.infer(training_config, drug=bortezomib, cell_line=cell_line)

In [ ]:
start_time = default_timer()
rec_pipeline.infer_embeddings(model_config)
print(f"***Took {default_timer() - start_time} seconds to save node embeddings.***")
rec_pipeline.graph.schema().view()

In [ ]:

#########################################################################


In [ ]:

#  if input_config.save_graph_path:
#      export_data.rdg(graph, input_config.save_graph_path)
    
    